In [13]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils.np_utils import to_categorical

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


# Load the dataset as a Pandas DataFrame
dataset = pd.read_csv('/content/sample_data/Sentiment (1).csv')

# Select only the necessary columns 'text' and 'sentiment'
mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]

# Keeping only the necessary columns
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [20]:
import tensorflow as tf
import logging

# Suppress TensorFlow logs
tf.get_logger().setLevel(logging.ERROR)

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets
    max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test data split
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 53s - loss: 0.8247 - accuracy: 0.6422 - 53s/epoch - 180ms/step
144/144 - 2s - loss: 0.7600 - accuracy: 0.6765 - 2s/epoch - 16ms/step
0.7600353360176086
0.6764962673187256


In [21]:
#1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

model.save('sentimentAnalysis.h3') #Saving the model

In [22]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h3') #loading the saved model

In [10]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [23]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")

1/1 - 0s - 305ms/epoch - 305ms/step
[0.6912919  0.14538696 0.16332114]
Neutral


In [24]:
#2. Apply GridSearchCV on the source code provided in the class

from keras.wrappers.scikit_learn import KerasClassifier #importing Keras classifier
from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

744/744 - 117s - loss: 0.8299 - accuracy: 0.6477 - 117s/epoch - 157ms/step
186/186 - 2s - loss: 0.7502 - accuracy: 0.6794 - 2s/epoch - 12ms/step
744/744 - 113s - loss: 0.8275 - accuracy: 0.6492 - 113s/epoch - 152ms/step
186/186 - 3s - loss: 0.7959 - accuracy: 0.6643 - 3s/epoch - 17ms/step
744/744 - 114s - loss: 0.8256 - accuracy: 0.6455 - 114s/epoch - 154ms/step
186/186 - 3s - loss: 0.7685 - accuracy: 0.6789 - 3s/epoch - 17ms/step
744/744 - 111s - loss: 0.8215 - accuracy: 0.6461 - 111s/epoch - 149ms/step
186/186 - 2s - loss: 0.7469 - accuracy: 0.6755 - 2s/epoch - 12ms/step
744/744 - 114s - loss: 0.8207 - accuracy: 0.6480 - 114s/epoch - 154ms/step
186/186 - 3s - loss: 0.7819 - accuracy: 0.6658 - 3s/epoch - 18ms/step
Epoch 1/2
744/744 - 112s - loss: 0.8270 - accuracy: 0.6470 - 112s/epoch - 151ms/step
Epoch 2/2
744/744 - 99s - loss: 0.6812 - accuracy: 0.7094 - 99s/epoch - 134ms/step
186/186 - 3s - loss: 0.7641 - accuracy: 0.6864 - 3s/epoch - 14ms/step
Epoch 1/2
744/744 - 113s - loss: 0.81